## First pratical of the Unsupervised Language Learning course 2017-2018

Joris Mollinga: 11871431 

Aron:

In [1]:
import numpy as np

In [ ]:
# Read the word embeddings file
fileName = 'deps.WORDS'

word2vec = {}

with open (fileName,'r',encoding="utf8") as emb:
    for line in emb:
        row = line.split()
        word = row[0]
        embedding = row[1:]
        
        # Convert to floats
        [float(i) for i in embedding]
        
        word2vec[word] = embedding       